
# Тестовое задание:
---

Требуется разработать модель, которая будет способна различать заголовки реальных и выдуманных новостей.

Для обучения модели используйте данные из файла `train.tsv`. В файле находится таблица, состоящая из двух колонок. 
В колонке title записан заголовок новости. В колонке is_fake содержатся метки: 0 – новость реальная; 1 – новость выдуманная.
Для демонстрации работы модели используйте данные тестового набора из файла `test.tsv`. В нем также есть колонка title, данные которой являются входными для вашей модели.

Вам нужно скопировать файл `test.tsv`, переименовать его в `predictions.tsv` и заполнить колонку is_fake значениями предсказаний вашей модели, аналогично `train.tsv`. 
Изначально колонка заполнена значением 0.

In [1]:
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import f1_score

rand_state = 42

In [2]:
train_data = pd.read_csv('dataset/train.tsv', sep = '\t')
#train_data.style.hide_index()
train_data.head(15)

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1
5,Российским студентам запретят учиться за рубежом,1
6,Путин пишет книгу об истории Украины,1
7,Россияне обхитрили рост цен,0
8,Звезда «Ворониных» раскрыл подробности о своем...,0
9,Microsoft объявила дату выхода очков дополненн...,0


---
# 1. Предобработка заголовков:

1) Из исходных заголовоков уберём все стоп-слова, несущие мало смысла (что, чтобы, как итд..)
- Имеет ли смысл также удалить цифры и прочие небуквенные символы? 
- А возможно и нет: для модели оказались довольно значимыми года (2022 итд)

2) Приведём каждое оставшееся слово к лемме (языковой единице в инфинитиве)

3) Применим подход TF-IDF


In [12]:
def preprocessing(df):
    
    '''df = any given DataFrame with title names'''
    
    len_df = len(df)
    
    # удаление стоп слов:
    stop_words = set(stopwords.words('russian'))
    
    for i in range(len_df):

        words = word_tokenize(df['title'][i])
        wordsFiltered = []

        for w in words:
            if w not in stop_words:
                wordsFiltered.append(w)

        df['title'][i] = " ".join(wordsFiltered)

        
    # лемантизация:
    morph = MorphAnalyzer()

    for i in range(len_df):

        title_words = word_tokenize(df['title'][i])
        lemmatized_title = []

        for word in title_words:

            p = morph.parse(word)[0]
            lemmatized_title.append(p.normal_form)

        df['title'][i] = " ".join(lemmatized_title)
        
    # В ходе исслаодвания также пробовал стеммер nltk.stem.snowball.SnowballStemmer, 
    # Он давал схожую точность, но не всегда верно обрабатывал слова (Навальный -> Навальн итд)
    
    return df

train_data = preprocessing(train_data)
train_data.head(15)

<ipython-input-12-e2379f475525>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = " ".join(wordsFiltered)
<ipython-input-12-e2379f475525>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = " ".join(lemmatized_title)


,title,is_fake
0,москвич владимир клутиный прийти счёт вмешател...,1
1,агент кокорин назвать езда встречка житейский ...,0
2,госдума рассмотреть возможность введение секре...,1
3,фас заблокировать поставка скоростной трамвай ...,0
4,против навальный завести дело недоносительство...,1
5,российский студент запретить учиться рубеж,1
6,путин писать книга история украина,1
7,россиянин обхитрить рост цена,0
8,звезда « воронин » раскрыть подробность свой с...,0
9,microsoft объявить дата выход очко дополнить р...,0


In [4]:
# TF-IDF (Токенизируем данные и строим словарь всех известных слов):
vectorizer = TfidfVectorizer()
train_x_vect = vectorizer.fit_transform(train_data['title'])
y_train = train_data['is_fake']

tmp = train_x_vect.toarray().shape
print('по всем {0} сообщениям набралось {1} уникальных слов'.format(tmp[0], tmp[1]))

по всем 5758 сообщениям набралось 11031 уникальных слов


# 2. Модель support vector machine (SVM):

In [5]:
# Метод опорных векоторов с линейным ядром - выберем наиболее оптимальный параметр перебором:

cv = KFold(n_splits = 5, shuffle = True, random_state = rand_state)
clf = SVC(kernel = 'linear', random_state = rand_state)

grid_svc = {'C': np.linspace(0.5, 2, 20)}
gs_svc = GridSearchCV(clf, grid_svc, scoring = 'f1', cv = cv)
gs_svc.fit(train_x_vect, y_train)

print('max f1 score: {}'.format(gs_svc.best_score_))
print('parameters for this score: {}\n'.format(gs_svc.best_params_))

max f1 score: 0.8475979580953513
parameters for this score: {'C': 0.8947368421052632}



 - Стоит ли пробовать нелинейное ядро?

In [6]:
# Лучшая модель:
svc_clf = SVC(kernel = 'linear', random_state = rand_state, C = 1) #gs_svc.best_params_['C'])
svc_clf.fit(train_x_vect, y_train)

# 20 самых значимых для моедли слов:
A = svc_clf.coef_.indices
B = np.abs(svc_clf.coef_.data)
most_important_words = A[B.argsort()[-20:][::-1]]

print('Наиболее значимые для модели слова: \n')
for i in most_important_words:
    print(vectorizer.get_feature_names()[i])

Наиболее значимые для модели слова: 

навальный
байден
гражданин
роскосмос
который
млн
нобелевский
рпц
лукашенко
запретить
нефть
дмитрий
сколково
россиянин
право
обязать
футболист
вакцина
выборы
миллиард


In [7]:
# Ответ на тестовых данных:

test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data = preprocessing(test_data)
test_data['is_fake'] = svc_clf.predict(vectorizer.transform(test_data['title']))

#test_data.style.hide_index()
test_data.head(10)

<ipython-input-3-7f8fc3c19b75>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = " ".join(wordsFiltered)
<ipython-input-3-7f8fc3c19b75>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = " ".join(lemmatized_title)


,title,is_fake
0,роскомнадзор представить реестр сочетание цвет...,1
1,ночью минский президентский гора беларашмор ( ...,1
2,бывший спичрайтер юрий лоза рассказать труднос...,1
3,"сельский церковь , собрать рекордно низкий кол...",1
4,акция google рухнуть объявление перезапуск rutube,0
5,курс доллар вырасти исторический максимум,0
6,опек назвать оптимальный уровень цена нефть,0
7,российский авиакомпания открыть рейс тбилиси урал,0
8,швейцарский горнолыжница расстрелять дом родитель,1
9,учредить театральный премия имя гарольд пинтер,0


Судя по поисковым запросам, есть подозрение, что большинство новостей на самом деле правдивые

Нужно ли уменьшать колличество ложноположительных ошибок (False-Positive rate)?

---
# 1.2. Модель random forest classifier (RFC):

In [8]:
# Выберем наиболее оптимальный параметр перебором:

cv = KFold(n_splits = 5, shuffle = True, random_state = rand_state)
clf = RFC(random_state = rand_state)

grid_rfc = {'max_depth': np.arange(5, 25)}
gs_rfc = GridSearchCV(clf, grid_rfc, scoring = 'f1', cv = cv)
gs_rfc.fit(train_x_vect, y_train)

print('max f1 score: {}'.format(gs_rfc.best_score_))
print('params for this score: {}\n'.format(gs_rfc.best_params_))

max f1 score: 0.7564930015720999

params for this score: {'max_depth': 22}



In [10]:
# Лучшая модель:
rfc_clf = RFC(max_depth = gs_rfc.best_params_['max_depth'], random_state = rand_state)
rfc_clf.fit(train_x_vect, y_train)

# Ответ на тестовых данных:
test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data = preprocessing(test_data)
test_data['is_fake'] = rfc_clf.predict(vectorizer.transform(test_data['title']))

#test_data.style.hide_index()
test_data.head(10)

<ipython-input-3-7f8fc3c19b75>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = " ".join(wordsFiltered)
<ipython-input-3-7f8fc3c19b75>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = " ".join(lemmatized_title)


,title,is_fake
0,роскомнадзор представить реестр сочетание цвет...,1
1,ночью минский президентский гора беларашмор ( ...,1
2,бывший спичрайтер юрий лоза рассказать труднос...,0
3,"сельский церковь , собрать рекордно низкий кол...",0
4,акция google рухнуть объявление перезапуск rutube,0
5,курс доллар вырасти исторический максимум,0
6,опек назвать оптимальный уровень цена нефть,0
7,российский авиакомпания открыть рейс тбилиси урал,0
8,швейцарский горнолыжница расстрелять дом родитель,1
9,учредить театральный премия имя гарольд пинтер,0


Судя по всему модель случайного леса хуже подходит для данной задачи

---
# 3.Вывод:
Наилучшую точность по метрике F1 показал подход TF_IDF + SVC 

In [11]:
test_data = pd.read_csv('dataset/test.tsv', sep = '\t')
test_data['is_fake'] = svc_clf.predict(vectorizer.transform(test_data['title']))
test_data.to_csv('predictions.tsv', index = False, sep = '\t')